<a href="https://colab.research.google.com/github/mnijhuis-dnb/Artificial_Intelligence_and_Machine_Learning_for_SupTech/blob/main/Tutorials/Tutorial%205%20Decision%20trees%20and%20random%20forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Artificial Intelligence and Machine Learning for SupTech  
Tutorial 5: Decision trees and random forests

*	Growing your own decision tree
*	How deep? How many splits? How big are the leaves?
*	From trees to random forests
*	Comparing performance with the confusion matrix

<br/>

14 March 2023  

**Instructors**  
Prof. Iman van Lelyveld (iman.van.lelyveld@vu.nl)<br/>
Dr. Michiel Nijhuis (m.nijhuis@dnb.nl)  

## Company performance
The data we are going to work with in this tutorial is data from the performance of companies over a 5 year period. We are going to see if we can predict the sector a company is operating in based on the performance data

In [ ]:
!gdown 1PCu4jNahysRpZ72z31KHpVkyAOp6nrKj

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Let's start by reading in the data

In [ ]:
df = pd.read_csv('/content/company_data.csv')

In [ ]:
df

In [ ]:
len(df_returns['permno'].unique())

In [ ]:
df_returns.head(5)

In [ ]:
df_returns.dtypes

Interestingly, `RET` is stored as object. This is problematic, since we need numeric values. Can you think of a reason why this column is loaded as string?
. <br/>
. <br/>
. <br/>
. <br/>
. <br/>
. <br/>
. <br/>

In [ ]:
df_returns['RET'].describe()

Evidently, the most common value `top` is a dot `.`. This is a marker for missing values. In python we use `np.nan` do designate a missing value, which is a numerical data type.

In [ ]:
df_returns['RET'] = df_returns['RET'].replace('.', np.nan)
df_returns['RET'] = df_returns['RET'].astype(float)

In [ ]:
df_returns['RET'].describe()

Note that the `date` column is an object. It is stored as a string. However, we may be interested in the date itself, i.e. the month or the year, such that we can compute monthly averages, changes over time, etc.


In [ ]:
df_returns['date'].head()

In [ ]:
df_returns['date'] = pd.to_datetime(df_returns['date'], format="%Y-%m-%d")
df_returns['date'].head()

In [ ]:
df_returns['date'].dt.hour

Now, let's tell python that this is a DataFrame that contains **panel** data, i.e. it has both an entity and date identifier. 

In Stata, this would be similar to `xtset`

In [ ]:
df_returns = df_returns.set_index(['permno','date'])

In [ ]:
df_returns.tail(30)

Doing so allows us to select and aggregate relevant subsets. For example, we want to plot the return series of company with id `11018`

In [ ]:
sr_11018 = df_returns.loc[11018,'RET']
sr_11018.head()

In [ ]:
sr_11018.plot(title='Returns of 11018');

What if we want to know the 1-year average return?

In [ ]:
sr_11018_avg = sr_11018.resample('Y').std()
sr_11018_avg.plot(title='Returns of 11018 (annual average)');

What if we want to know the 1-year moving average?

In [ ]:
sr_11018_ma = sr_11018.rolling(12).mean()
sr_11018_ma.plot(title='Returns of 11018 (1-year MA)');

Or the 3-month rolling standard deviation?

In [ ]:
sr_11018_sd = sr_11018.rolling(3).std()
sr_11018_sd.plot(title='Returns of 11018 (1-year SD)');

## Firm characteristics

`characteristics.csv` contains `df_features` Column 1 (`permno`) is the unique company identifier.
Column 2 (`date`) is the unique date identifier.
Column 3-96 are the 94 lagged firm characteristics.
We will refer to these as $X_{it}$. For more information, have a look at Green, Hand, Zhang (2017), in the literature folder.


In [ ]:
path = 'company.csv'
df_features = pd.read_csv(path)

df_features['date'] = pd.to_datetime(df_features['date'])
df_features = df_features.set_index(['permno','date'])

In [ ]:
df_features.dtypes

In [ ]:
df_features.describe().T

In [ ]:
df_features.head()

## Missing values

## Over time

In [ ]:
df_returns.notna()

In [ ]:
df_returns.notna().groupby('date').mean()

In [ ]:
df_returns.notna().groupby('date').sum().plot()

## Fill missing values for companies

### Returns
After having a look at how severe the problem is, we decide to fill in those missing with the value `0`. In other words, we assume that the companies did not have any stock appreciation or depreciation in that month.

In [ ]:
sr_nb_missing = df_returns.notna().groupby('permno').sum()

In [ ]:
sr_nb_missing.value_counts().sort_values()

In [ ]:
df_returns = df_returns.fillna(0)

### Firm characteristics
In this case, for each characteristic we fill in the `median` over all companies.

In [ ]:
df_features

In [ ]:
df_features.dropna(axis=0).shape

In [ ]:
df_features.dropna(axis=1).shape

In [ ]:
df_features.dropna(how='all', axis=0).shape

In [ ]:
sr_medians = df_features.median()
sr_medians.head()

In [ ]:
df_features = df_features.fillna(sr_medians)

In [ ]:
df_features.notna().mean().mean()

### Ensure that returns and characteristics cover the same firms and periods

In [ ]:
df_returns.index.shape

In [ ]:
df_features.index.shape

In [ ]:
df_returns.index

In [ ]:
df_features.index

In [ ]:
common_index = df_returns.index.intersection(df_features.index)
common_index

In [ ]:
common_index.shape

In [ ]:
df_returns = df_returns.loc[common_index]
df_features = df_features.loc[common_index]

# What characteristics explain stock returns?

What determines the differences between average stock returns? This perspective implies that we do not care much about the time-variation. If this were a linear regression model, you would estimate

$$
r_i = X_i\beta + e_i
$$

where $r_i$ is the average return over the entire observation sample. Similarly, $X_i$ is the average characteristic of firm $i$.

In [ ]:
import statsmodels.api as sm

In [ ]:
df_returns

In [ ]:
sr_targets = df_returns.groupby('permno').mean()
sr_targets.head()

In [ ]:
df_features = df_features.groupby('permno').mean()
df_features.head()

In [ ]:
df_features.corr()

Let us only select those with low correlation.

In [ ]:
sr_corr = df_features.corr().abs().stack().sort_values()
sr_corr = sr_corr[sr_corr<1]
sr_corr = sr_corr.groupby(level=0).max()
sr_corr.sort_values()

df_features = df_features.loc[:, sr_corr < .7]
df_features.shape

We can now fit a regular OLS model

In [ ]:
ols = sm.OLS(sr_targets, df_features).fit()

In [ ]:
ols.summary()

However, with 93 predictors, it is hard to see what the main drivers are. Plus, we have strong multicollinarity issues.

# Regulariztion

Lasso ($L_1$-norm)
<img src="http://dieter.wang/files/images/l1-reg.png" />

In [ ]:
from sklearn.linear_model import Lasso

Let us start with an arbitrary penalty `alpha = .001`

In [ ]:
alpha = .001

In [ ]:
lasso = Lasso(
    alpha=alpha,
    fit_intercept=True
)

In [ ]:
lasso.fit(
    df_features,
    sr_targets, 
)

In [ ]:
lasso.coef_.round(3)

In [ ]:
sr_coef = pd.Series(lasso.coef_, index=df_features.columns)

In [ ]:
marker = sr_coef != 0
sum(marker)

In [ ]:
sr_coef.loc[marker]

## Is this a fair competition?

Regularization strongly depends on the scaling of the features. We can see that they are very heterogenous. Those with higher variance will be penalized more.

In [ ]:
df_features.std()

In [ ]:
df_features.std().sort_values()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

df_features_rescaled = pd.DataFrame(
    scaler.fit_transform(df_features),
    index=df_features.index,
    columns=df_features.columns,
)

In [ ]:
df_features_rescaled.mean()

In [ ]:
df_features_rescaled.std()

Let us now re-fit the Lasso with the rescaled `df_features`

In [ ]:
lasso.fit(
    df_features_rescaled,
    sr_targets, 
)

In [ ]:
sr_coef = pd.Series(lasso.coef_, index=df_features.columns)
sr_coef.abs().sort_values()

In [ ]:
sum(sr_coef != 0)

## What is the right penalty?

For illustration, we just used `alpha=` to show the issue of a "non-level playing field". But how would we select the ideal penalty? By what criterion should this be done?

In [ ]:
from sklearn.linear_model import lasso_path

In [ ]:
np.logspace(-5,-1,30)

In [ ]:
lasso_alphas, lasso_coefs, _ = lasso_path(
    df_features_rescaled, 
    sr_targets, 
    alphas=np.logspace(-5,-1,30)
)
lasso_logalphas = np.log10(lasso_alphas)

In [ ]:
fig, ax = plt.subplots(figsize=[16,9])
ax.plot(lasso_logalphas, lasso_coefs.squeeze().T)
ax.set_title('Lasso path')
ax.set_xlabel('< low penalty | high penalty >')
plt.grid()

## K-fold cross-validation

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" />

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold

In [ ]:
cv = KFold(n_splits=10)

In [ ]:
lassocv = LassoCV(fit_intercept=True, cv=cv, n_alphas=100)
result = lassocv.fit(df_features_rescaled, sr_targets)

In [ ]:
fig = plt.figure(figsize=[16,9])
plt.plot(np.log10(result.alphas_), result.mse_path_ - result.mse_path_.mean(axis=0, keepdims=True))
plt.axvline(np.log10(result.alpha_))
plt.title('MSE across different log10(alpha) for 5-fold cross-validation')
plt.grid()

In [ ]:
sr_coef = pd.Series(lassocv.coef_, index=df_features.columns)
sr_coef[sr_coef != 0].sort_values()

## Other types of cross-validation techniques

Different problems, data availability require different cross-validation methods. We discuss a few here. For a comprehensive overview, see [sklearn manual](https://scikit-learn.org/stable/modules/cross_validation.html)

### Leave One Out
```
fold 0   x . . . . . . . . .
fold 1   . x . . . . . . . .
 ...
fold N   . . . . . . . . . x
```
Pros: ?  
Cons: ?

### Stratified K-Fold
<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png" />

Pros: ?  
Cons: ?

### Timeseries Split
<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png" />

Pros: ?  
Cons: ?


### hv-block Cross Validation
<img src="https://images.deepai.org/converted-papers/1910.08904/images/hv-block.png"/>

Pros: ?  
Cons: ?

### Group K-fold
<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_007.png">

Pros: ?  
Cons: ?

# Decision trees and random forests

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_tree_regression_001.png" />

We now explore decision trees and their generalization, random forests. Trees are very powerful but tend to overfit. Yesterday, we looked at how their recall and precision fared when allowing for `max_depth` to change.

Today we use the classification report to compare decision trees, random forests and support vector machines on **out-of-sample** performances.

For this we do the following for each model

1. Split the sample into training and testing
2. Pick hyperparameters using cross-validation in the training
3. Make predictions for the testing data

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

Splitting into training and testing

In [ ]:
X = df_features.values 
y = sr_targets.squeeze().values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=.2,
    shuffle=True,
    random_state=1,
)

## Decision tree

In [ ]:
mdl_tree = DecisionTreeRegressor()

In [ ]:
gscv_tree = GridSearchCV(
    estimator=mdl_tree,
    param_grid=dict(
        max_depth=np.arange(1,20),
    ),
    cv=KFold(n_splits=5),
)

In [ ]:
gscv_tree.fit(X_train, y_train)

In [ ]:
gscv_tree.cv_results_

In [ ]:
optm_tree = gscv_tree.best_estimator_

In [ ]:
optm_tree.max_depth

In [ ]:
nonzero = optm_tree.feature_importances_ != 0

In [ ]:
df_features.columns[nonzero]

In [ ]:
nonzero = optm_tree.feature_importances_ != 0

In [ ]:
y_test_pred = optm_tree.predict(X_test)
y_train_pred = optm_tree.predict(X_train)

mse_tree_test = mean_squared_error(y_test_pred, y_test)
mse_tree_train = mean_squared_error(y_train_pred, y_train)

In [ ]:
mse_tree_test

In [ ]:
mse_tree_train

## Random Forest

<img src="https://upload.wikimedia.org/wikipedia/commons/7/76/Random_forest_diagram_complete.png" />

In [ ]:
mdl_forest = RandomForestRegressor(
    n_estimators=5
)

In [ ]:
gscv_forest = GridSearchCV(
    estimator=mdl_forest,
    param_grid=dict(
        max_depth=np.arange(1,20),
    ),
    cv=KFold(n_splits=10),
)

In [ ]:
gscv_forest.fit(X_train, y_train)

In [ ]:
optm_forest = gscv_forest.best_estimator_

In [ ]:
optm_forest.n_estimators

In [ ]:
optm_forest.max_depth

In [ ]:
y_test_pred = optm_forest.predict(X_test)
y_train_pred = optm_forest.predict(X_train)
mse_forest_test = mean_squared_error(y_test_pred, y_test)
mse_forest_train = mean_squared_error(y_train_pred, y_train)

In [ ]:
mse_forest_test

In [ ]:
mse_forest_train

## Support Vector Regression


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Svr_epsilons_demo.svg/2880px-Svr_epsilons_demo.svg.png" />

In [ ]:
mdl_svr = SVR()

In [ ]:
gscv_svr = GridSearchCV(
    estimator=mdl_svr,
    param_grid=dict(
        kernel=['linear','rbf'],
        C=np.linspace(.1,2,10),
    ),
    cv=KFold(n_splits=10),
)

In [ ]:
gscv_svr.fit(X_train, y_train)

In [ ]:
optm_svr = gscv_svr.best_estimator_

In [ ]:
y_test_pred = optm_svr.predict(X_test)
y_train_pred = optm_svr.predict(X_train)
mse_svr_test = mean_squared_error(y_test_pred, y_test)
mse_svr_train = mean_squared_error(y_train_pred, y_train)

In [ ]:
mse_svr_test

In [ ]:
mse_svr_train

## Comparison

In [ ]:
print('mse_tree_test   ', mse_tree_test)
print('mse_tree_train  ', mse_tree_train)
print('mse_forest_test ', mse_forest_test)
print('mse_forest_train', mse_forest_train)
print('mse_svr_test    ', mse_svr_test)
print('mse_svr_train   ', mse_svr_train)

# Clustering

We try to cluster firms based on their characteristics. Note that we do not use the returns here, as this is **unsupervised learning**. I.e. we are not interested in clustering such that we obtain better predictions. Instead, we want to best "summarize" the data for the "general reader". 

See PCA and Dynamic factor models.

Full documentation here [sklearn manual](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)

In [ ]:
x = 'aeavol'
y = 'rd'

In [ ]:
df_X = np.sqrt(df_features.loc[:,[x,y]])
df_X = df_X.dropna()

In [ ]:
df_X.plot.scatter(x,y)

In [ ]:
n_clusters = 3
colors = ['red','blue','green']

## KMeans clustering

In [ ]:
from sklearn.cluster import KMeans

We first estimate the full model using `KMeans`

In [ ]:
kmeans = KMeans(
    n_clusters=n_clusters, 
    random_state=0,
).fit(df_X)

Results are the cluster memberships of each company

In [ ]:
sr_clusters = pd.Series(kmeans.labels_, index=df_X.index, name='clusters')
sr_clusters

In [ ]:
sr_clusters.value_counts()

In [ ]:
df_X.join(sr_clusters).groupby('clusters').mean()

In [ ]:
df_X_clusters = df_X.join(sr_clusters).groupby('clusters')
centroids = df_X_clusters.mean().values

In [ ]:
# store for comparison
sr_clusters_km = sr_clusters.copy()

In [ ]:
for permno, (x1, x2) in df_X.iterrows():
  cluster = sr_clusters[permno]
  color = colors[cluster]
  plt.plot(
    x1, x2,
    color=color,
    marker='o', markersize=3, lw=0
  )
for i, (x1, x2) in enumerate(centroids):
  plt.plot(x1, x2, color=colors[i], marker='X', markersize=40, alpha=.6)

## AgglomerativeClustering

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_dendrogram_001.png">

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agg = AgglomerativeClustering(
    n_clusters=n_clusters
).fit(df_X)

sr_clusters = pd.Series(agg.labels_, index=df_X.index, name='clusters')
df_X_clusters = df_X.join(sr_clusters).groupby('clusters')
centers = df_X_clusters.mean().values

In [ ]:
# store for comparison
sr_clusters_agg = sr_clusters.copy()

In [ ]:
for permno, (x1, x2) in df_X.iterrows():
  cluster = sr_clusters[permno]
  color = colors[cluster]
  plt.plot(
    x1, x2,
    color=color,
    marker='o', markersize=3, lw=0
  )
for i, (x1, x2) in enumerate(centers):
  plt.plot(x1, x2, color=colors[i], marker='X', markersize=40, alpha=.6)

## How to evaluate clustering performance?

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:

help(silhouette_score)

## KMeans silhouette

In [ ]:
fig, axes = plt.subplots(1, n_clusters, sharey=True, figsize=[15,5])

silhouette_values = silhouette_samples(df_X, sr_clusters_km)

for cluster in sr_clusters.unique():
  ax = axes[cluster]
  color = colors[cluster]
  sils = silhouette_values[sr_clusters == cluster]
  sils = sorted(sils)
  ax.bar(range(len(sils)), sils, color=color, width=2)

  sils_avg = np.mean(sils)
  ax.axhline(sils_avg, lw=3, ls='--', color=color)
  ax.set_title(f'Cluster {cluster}\n(avg. silhoutte: {sils_avg:.3f}')

fig.tight_layout()

## Agglomerative clustering

In [ ]:
fig, axes = plt.subplots(1, n_clusters, sharey=True, figsize=[15,5])

silhouette_values = silhouette_samples(df_X, sr_clusters_agg)

for cluster in sr_clusters.unique():
  ax = axes[cluster]
  color = colors[cluster]
  sils = silhouette_values[sr_clusters == cluster]
  sils = sorted(sils)
  ax.bar(range(len(sils)), sils, color=color, width=2)

  sils_avg = np.mean(sils)
  ax.axhline(sils_avg, lw=3, ls='--', color=color)
  ax.set_title(f'Cluster {cluster}\n(avg. silhoutte: {sils_avg:.3f}')

fig.tight_layout()